# CmdStanPy Example Notebook

In [1]:
# Load packages used in this notebook
import os
import json
import pandas as pd

Step 1: install CmdStanPy

In [2]:
# Install package CmdStanPy
!pip install cmdstanpy

You should consider upgrading via the '/Users/alp/github/probprog-finalproject/venv/bin/python -m pip install --upgrade pip' command.


Step 2: install Stan itself

In [3]:
# Check CmdStan path
from cmdstanpy import CmdStanModel, cmdstan_path, install_cmdstan
install_cmdstan()

Installing CmdStan version: 2.27.0
Install directory: /Users/alp/.cmdstan
CmdStan version 2.27.0 already installed
deleting tmpfiles dir: /var/folders/zg/zlpj91s13w7566wy5qd0lg4w0000gn/T/tmpzta1adq5
done



True

Step 3: make sure path is `~/.cmdstan`

In [4]:
cmdstan_path()

'/Users/alp/.cmdstan/cmdstan-2.27.0'

The CmdStan installation includes a simple example program `bernoulli.stan` and test data `bernoulli.data.json`. These are in the CmdStan installation directory `examples/bernoulli`.

The program `bernoulli.stan` takes a vector `y` of length `N` containing binary outcomes and uses a bernoulli distribution to estimate `theta`, the chance of success.

In [5]:
bernoulli_stan = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')
with open(bernoulli_stan, 'r') as fd:
        print('\n'.join(fd.read().splitlines()))

data {
  int<lower=0> N;
  array[N] int<lower=0,upper=1> y; // or int<lower=0,upper=1> y[N];
}
parameters {
  real<lower=0,upper=1> theta;
}
model {
  theta ~ beta(1,1);  // uniform prior on interval 0,1
  y ~ bernoulli(theta);
}


The data file `bernoulli.data.json` contains 10 observations, split between 2 successes (1) and 8 failures (0).

In [6]:
bernoulli_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')
with open(bernoulli_data, 'r') as fd:
        print('\n'.join(fd.read().splitlines()))

{
    "N" : 10,
    "y" : [0,1,0,0,0,0,0,0,0,1]
}


The following code test that the CmdStanPy toolchain is properly installed by compiling the example model, fitting it to the data, and obtaining a summary of estimates of the posterior distribution of all parameters and quantities of interest.



In [7]:
# Run CmdStanPy Hello, World! example
from cmdstanpy import cmdstan_path, CmdStanModel

# Compile example model bernoulli.stan
bernoulli_model = CmdStanModel(stan_file=bernoulli_stan)

# Condition on example data bernoulli.data.json
bern_fit = bernoulli_model.sample(data=bernoulli_data, seed=123)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/alp/.cmdstan/cmdstan-2.27.0/examples/bernoulli/bernoulli
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


In [8]:
# Print a summary of the posterior sample
bern_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-7.30,0.0170,0.71,-8.70,-7.00,-6.80,1700.0,29000.0,1.0
theta,0.25,0.0029,0.12,0.08,0.23,0.47,1700.0,29000.0,1.0
